In [71]:
%run SetUpParams.ipynb

2024-08-12 01:59:21.668766
2004-08-12 01:59:21.668766


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [72]:
import pandas as pd 
import numpy as np

### Create dataframe to feed into model

In [73]:
final_df = pd.DataFrame()
# Next Day Open Price - Current Day Open Price
final_df[f'{target_stock}_change'] = target_stock_data['Open'].shift(-1) - target_stock_data['Open']
final_df[f'{target_stock}_lag'] = final_df[f'{target_stock}_change'].shift(1)
final_df['Close'] = target_stock_data['Close']
final_df['Price'] = target_stock_data['Open']
final_df.reset_index(inplace=True)
print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price
0    2004-08-12    -12.560059         NaN  1063.229980  1075.790039
1    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980
2    2004-08-16     14.539917    1.570068  1079.339966  1064.800049
3    2004-08-17      2.369995   14.539917  1081.709961  1079.339966
4    2004-08-18     13.460083    2.369995  1095.170044  1081.709961
...         ...           ...         ...          ...          ...
5028 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137
5029 2024-08-06     86.709961   55.279785  5240.029785  5206.419922
5030 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883
5031 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824
5032 2024-08-09           NaN   62.090332  5344.160156  5314.660156

[5033 rows x 5 columns]


#### Add European and US Stock Market Price Changes to final_df
Current Day Open Price - Previous Day Open Price

In [74]:
# EU Stock Market has already opened
for ticker in US_EU_market_dict.keys():
        data = US_EU_market_dict[ticker]
        # Current Day Open Price - Previous Day Open Price
        temp = (data['Open'] - data['Open'].shift(1)).reset_index()
        temp.rename(columns={'Open':ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price  \
0    2004-08-12    -12.560059         NaN  1063.229980  1075.790039   
1    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980   
2    2004-08-16     14.539917    1.570068  1079.339966  1064.800049   
3    2004-08-17      2.369995   14.539917  1081.709961  1079.339966   
4    2004-08-18     13.460083    2.369995  1095.170044  1081.709961   
...         ...           ...         ...          ...          ...   
5028 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137   
5029 2024-08-06     86.709961   55.279785  5240.029785  5206.419922   
5030 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883   
5031 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824   
5032 2024-08-09           NaN   62.090332  5344.160156  5314.660156   

           ^GSPC        ^IXIC         ^DJI       ^FCHI      ^GDAXI  
0            NaN          NaN          NaN         NaN         NaN  
1     -12

#### Add Asian Stock Market Price Changes to final_df
Close Price - Open Price

In [75]:
# Asian Stock Market has already closed; one day ahead of US Stock Market
for ticker in asia_market_dict.keys():
        data = asia_market_dict[ticker]
        # Close Price - Open Price
        temp = (data['Close'] - data['Open']).reset_index()
        temp.rename(columns={0:ticker}, inplace=True)
        final_df = final_df.merge(temp, on='Date', how='left')

print(final_df)

           Date  ^GSPC_change   ^GSPC_lag        Close        Price  \
0    2004-08-12    -12.560059         NaN  1063.229980  1075.790039   
1    2004-08-13      1.570068  -12.560059  1064.800049  1063.229980   
2    2004-08-16     14.539917    1.570068  1079.339966  1064.800049   
3    2004-08-17      2.369995   14.539917  1081.709961  1079.339966   
4    2004-08-18     13.460083    2.369995  1095.170044  1081.709961   
...         ...           ...         ...          ...          ...   
5028 2024-08-05     55.279785 -225.489746  5186.330078  5151.140137   
5029 2024-08-06     86.709961   55.279785  5240.029785  5206.419922   
5030 2024-08-07    -40.560059   86.709961  5199.500000  5293.129883   
5031 2024-08-08     62.090332  -40.560059  5319.310059  5252.569824   
5032 2024-08-09           NaN   62.090332  5344.160156  5314.660156   

           ^GSPC        ^IXIC         ^DJI       ^FCHI      ^GDAXI  \
0            NaN          NaN          NaN         NaN         NaN   
1     -

In [77]:
# Different markets has different holidays
# Forward fill null values
final_df.ffill(inplace=True)
final_df.dropna(inplace=True)
# final_df.isna().sum()